In [7]:
!nvidia-smi

Wed Dec  9 19:51:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 27%   32C    P8    25W / 260W |   5990MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 31%   45C    P2    60W / 260W |   6938MiB / 11019MiB |      7%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [8]:
import sys
import datetime


import torch
import numpy as np
from pathlib import Path



from torch.utils.data import DataLoader
from allennlp.data import allennlp_collate
from allennlp.training.util import evaluate
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.data.dataset_readers.sequence_tagging import SequenceTaggingDatasetReader
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.token_indexers import PretrainedTransformerMismatchedIndexer

from allennlp.nn.initializers import PretrainedModelInitializer, InitializerApplicator


from transformers import RobertaModel

seed=1
torch.manual_seed(seed)
np.random.seed(seed)

In [9]:
cuda_device = torch.device('cuda:5')
n_gpu = torch.cuda.device_count()

for i in range(n_gpu):
    print(torch.cuda.get_device_name(i))

GeForce RTX 2080 Ti
GeForce RTX 2080 Ti
GeForce RTX 2080 Ti
GeForce RTX 2080 Ti
GeForce RTX 2080 Ti
GeForce RTX 2080 Ti
GeForce RTX 2080 Ti
GeForce RTX 2080 Ti


In [10]:
data_folder = './data'
pretrained_roberta_bin = '../toxicity/models/roberta_base/pytorch_model.bin'
pretrained_roberta_folder = '../toxicity/models/roberta_base'
ROBERTA_MODEL = 'roberta-base'
LOG_PATH=Path('../logs/')
LOG_PATH.mkdir(exist_ok=True)
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

In [11]:
import logging

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, 'TEST'))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

In [12]:
def get_parameter_overrides():
    from allennlp.models import SimpleTagger
    model = SimpleTagger(text_field_embedder=text_field_embedder, 
                      vocab=vocab, 
                      encoder=seq2seq_encoder,
                      calculate_span_f1=True,
                      label_encoding='IOB1').cuda(device=cuda_device)
    pretrained_roberta = PretrainedModelInitializer(pretrained_roberta_bin)
    allen_layer_names = list(model.state_dict().keys())
    trans_layer_names = list(pretrained_roberta.weights.keys())
    allen_fig = allen_layer_names[:len(trans_layer_names)]
    trans_fig = trans_layer_names
    overrides_dict = dict(zip(allen_fig, trans_fig))
    positions = '|'.join(allen_fig)
    return overrides_dict, positions

## Model

In [13]:
indexer = PretrainedTransformerMismatchedIndexer(model_name=ROBERTA_MODEL)

In [14]:
wordpieces, offsets = indexer._allennlp_tokenizer.intra_word_tokenize(['We', 'are', 'living', 'in', 'Mönchengladbach'])
wordpieces

[<s>, We, are, living, in, M, Ã¶n, che, ng, lad, bach, </s>]

In [15]:
reader = SequenceTaggingDatasetReader(token_indexers={'tokens' : indexer})
train_dataset = reader.read('data/train.seq')
dev_dataset = reader.read('data/dev.seq')
test_dataset = reader.read('data/trial.seq')

12/09/2020 19:51:10 - INFO - allennlp.data.dataset_readers.sequence_tagging -   Reading instances from lines in file at: data/train.seq


12/09/2020 19:51:11 - INFO - allennlp.data.dataset_readers.sequence_tagging -   Reading instances from lines in file at: data/dev.seq


12/09/2020 19:51:11 - INFO - allennlp.data.dataset_readers.sequence_tagging -   Reading instances from lines in file at: data/trial.seq


In [16]:
vocab = Vocabulary.from_instances(train_dataset.instances)
train_dataset.index_with(vocab)
dev_dataset.index_with(vocab)
test_dataset.index_with(vocab)

12/09/2020 19:51:12 - INFO - allennlp.data.vocabulary -   Fitting token dictionary from dataset.


In [17]:
from allennlp.modules.token_embedders import PretrainedTransformerMismatchedEmbedder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.seq2seq_encoders import PassThroughEncoder


embedder = PretrainedTransformerMismatchedEmbedder(model_name=ROBERTA_MODEL)
text_field_embedder = BasicTextFieldEmbedder({'tokens': embedder})
seq2seq_encoder = PassThroughEncoder(input_dim=embedder.get_output_dim())

In [ ]:
parameter_name_overrides, positions = get_parameter_overrides()

In [19]:
pretrained_roberta = PretrainedModelInitializer(pretrained_roberta_bin, 
                                                parameter_name_overrides)

In [20]:
initializer = InitializerApplicator([(positions, pretrained_roberta)])

In [ ]:
from allennlp.models import SimpleTagger


model = SimpleTagger(text_field_embedder=text_field_embedder, 
                      vocab=vocab, 
                      encoder=seq2seq_encoder,
                      calculate_span_f1=True,
                      label_encoding='IOB1',
                     initializer=initializer).cuda(device=cuda_device)

In [22]:
import torch.optim as optim
from transformers import AdamW
import transformers
from datetime import datetime

from allennlp.training.learning_rate_schedulers import LinearWithWarmup
from torch.utils.data import DataLoader
from allennlp.training import GradientDescentTrainer
from allennlp.training.learning_rate_schedulers import SlantedTriangular

import math


num_epochs = 10
batch_size = 10
steps_per_epoch = math.ceil(len(train_dataset) / batch_size)

optimizer = optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, 
                               collate_fn=allennlp_collate, shuffle=True)
val_data_loader = DataLoader(dataset=dev_dataset, batch_size=50, collate_fn=allennlp_collate)
lr_scheduler = LinearWithWarmup(optimizer, 
                                num_epochs=num_epochs, 
                                warmup_steps=(steps_per_epoch*num_epochs)*0.1, 
                                num_steps_per_epoch=steps_per_epoch)

date_time = datetime.now()
date_str = date_time.strftime('%m/%d/%Y')
time_str = date_time.strftime('%H:%M:%S')


trainer = GradientDescentTrainer(
    model=model,
    optimizer=optimizer,
    data_loader=train_data_loader,
    validation_data_loader=val_data_loader,
    num_epochs=num_epochs,
    cuda_device=cuda_device,
    learning_rate_scheduler=lr_scheduler,
    num_gradient_accumulation_steps=1,
    serialization_dir=f'./workdir/{date_str}/{time_str}',
    grad_clipping=1.
)

try:
    metrics = trainer.train()
except KeyboardInterrupt:
    pass

12/09/2020 19:51:23 - WARNING - allennlp.training.trainer -   You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
12/09/2020 19:51:23 - INFO - allennlp.training.trainer -   Beginning training.
12/09/2020 19:51:23 - INFO - allennlp.training.trainer -   Epoch 0/9
12/09/2020 19:51:23 - INFO - allennlp.training.trainer -   Worker 0 memory usage MB: 4458.972
12/09/2020 19:51:23 - INFO - allennlp.training.trainer -   GPU 0 memory usage MB: 7509
12/09/2020 19:51:23 - INFO - allennlp.training.trainer -   GPU 1 memory usage MB: 6938
12/09/2020 19:51:23 - INFO - allennlp.training.trainer -   GPU 2 memory usage MB: 9528
12/09/2020 19:51:23 - INFO - allennlp.training.trainer -   GPU 3 memory usage MB: 966
12/09/2020 19:51:23 - INFO - allennlp.training.trainer -   GPU 4 memory usage MB: 1064
12/09/2020 19:51:23 - INFO - allennlp.training.trainer -   GPU 5 memory usage MB: 1550
12/09/2020 19:51:23 - INFO - allennlp.training.trainer -   GPU 6 memory

12/09/2020 19:53:08 - INFO - allennlp.training.trainer -   Validating


12/09/2020 19:53:20 - INFO - allennlp.training.tensorboard_writer -                          Training |  Validation
12/09/2020 19:53:20 - INFO - allennlp.training.tensorboard_writer -   accuracy           |     0.933  |     0.940
12/09/2020 19:53:20 - INFO - allennlp.training.tensorboard_writer -   accuracy3          |     1.000  |     1.000
12/09/2020 19:53:20 - INFO - allennlp.training.tensorboard_writer -   f1-measure-overall |     0.335  |     0.493
12/09/2020 19:53:20 - INFO - allennlp.training.tensorboard_writer -   gpu_0_memory_MB    |  7509.000  |       N/A
12/09/2020 19:53:20 - INFO - allennlp.training.tensorboard_writer -   gpu_1_memory_MB    |  6938.000  |       N/A
12/09/2020 19:53:20 - INFO - allennlp.training.tensorboard_writer -   gpu_2_memory_MB    |  9528.000  |       N/A
12/09/2020 19:53:20 - INFO - allennlp.training.tensorboard_writer -   gpu_3_memory_MB    |   966.000  |       N/A
12/09/2020 19:53:20 - INFO - allennlp.training.tensorboard_writer -   gpu_4_memory_MB 

12/09/2020 19:53:23 - INFO - allennlp.training.checkpointer -   Best validation performance so far. Copying weights to './workdir/12/09/2020/19:51:23/best.th'.
12/09/2020 19:53:23 - INFO - allennlp.training.trainer -   Epoch duration: 0:02:00.420199
12/09/2020 19:53:23 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:18:03
12/09/2020 19:53:23 - INFO - allennlp.training.trainer -   Epoch 1/9
12/09/2020 19:53:23 - INFO - allennlp.training.trainer -   Worker 0 memory usage MB: 4945.86
12/09/2020 19:53:24 - INFO - allennlp.training.trainer -   GPU 0 memory usage MB: 7509
12/09/2020 19:53:24 - INFO - allennlp.training.trainer -   GPU 1 memory usage MB: 6938
12/09/2020 19:53:24 - INFO - allennlp.training.trainer -   GPU 2 memory usage MB: 9528
12/09/2020 19:53:24 - INFO - allennlp.training.trainer -   GPU 3 memory usage MB: 966
12/09/2020 19:53:24 - INFO - allennlp.training.trainer -   GPU 4 memory usage MB: 1084
12/09/2020 19:53:24 - INFO - allennlp.training.train

12/09/2020 19:54:59 - INFO - allennlp.training.trainer -   Validating


...

12/09/2020 20:07:37 - INFO - allennlp.training.tensorboard_writer -                          Training |  Validation
12/09/2020 20:07:37 - INFO - allennlp.training.tensorboard_writer -   accuracy           |     0.969  |     0.931
12/09/2020 20:07:37 - INFO - allennlp.training.tensorboard_writer -   accuracy3          |     1.000  |     1.000
12/09/2020 20:07:37 - INFO - allennlp.training.tensorboard_writer -   f1-measure-overall |     0.649  |     0.501
12/09/2020 20:07:37 - INFO - allennlp.training.tensorboard_writer -   gpu_0_memory_MB    |  7509.000  |       N/A
12/09/2020 20:07:37 - INFO - allennlp.training.tensorboard_writer -   gpu_1_memory_MB    |  6938.000  |       N/A
12/09/2020 20:07:37 - INFO - allennlp.training.tensorboard_writer -   gpu_2_memory_MB    |  9528.000  |       N/A
12/09/2020 20:07:37 - INFO - allennlp.training.tensorboard_writer -   gpu_3_memory_MB    |   966.000  |       N/A
12/09/2020 20:07:37 - INFO - allennlp.training.tensorboard_writer -   gpu_4_memory_MB 

12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   Epoch duration: 0:01:45.907423
12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:01:48
12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   Epoch 9/9
12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   Worker 0 memory usage MB: 4946.624
12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   GPU 0 memory usage MB: 7509
12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   GPU 1 memory usage MB: 6938
12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   GPU 2 memory usage MB: 9528
12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   GPU 3 memory usage MB: 966
12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   GPU 4 memory usage MB: 1046
12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   GPU 5 memory usage MB: 10698
12/09/2020 20:07:40 - INFO - allennlp.training.trainer -   GPU 6 memory usage MB: 11
12/09/2020 20:07:40 - INFO - allennlp.

12/09/2020 20:09:11 - INFO - allennlp.training.trainer -   Validating


12/09/2020 20:09:20 - INFO - allennlp.training.tensorboard_writer -                          Training |  Validation
12/09/2020 20:09:20 - INFO - allennlp.training.tensorboard_writer -   accuracy           |     0.971  |     0.931
12/09/2020 20:09:20 - INFO - allennlp.training.tensorboard_writer -   accuracy3          |     1.000  |     1.000
12/09/2020 20:09:20 - INFO - allennlp.training.tensorboard_writer -   f1-measure-overall |     0.666  |     0.502
12/09/2020 20:09:20 - INFO - allennlp.training.tensorboard_writer -   gpu_0_memory_MB    |  7509.000  |       N/A
12/09/2020 20:09:20 - INFO - allennlp.training.tensorboard_writer -   gpu_1_memory_MB    |  6938.000  |       N/A
12/09/2020 20:09:20 - INFO - allennlp.training.tensorboard_writer -   gpu_2_memory_MB    |  9528.000  |       N/A
12/09/2020 20:09:20 - INFO - allennlp.training.tensorboard_writer -   gpu_3_memory_MB    |   966.000  |       N/A
12/09/2020 20:09:20 - INFO - allennlp.training.tensorboard_writer -   gpu_4_memory_MB 

12/09/2020 20:09:23 - INFO - allennlp.training.trainer -   Epoch duration: 0:01:43.158295
12/09/2020 20:09:23 - INFO - allennlp.training.checkpointer -   loading best weights


In [23]:
torch.save(model.state_dict(), '../toxicity/models/roberta_spans')

In [24]:
test_dataloader = DataLoader(test_dataset, batch_size=100, collate_fn=allennlp_collate)
evaluate(model, test_dataloader, cuda_device=cuda_device)

12/09/2020 20:09:25 - INFO - allennlp.training.util -   Iterating over dataset


{'accuracy': 0.9514164305949009,
 'accuracy3': 1.0,
 'precision-overall': 0.6384039900249375,
 'recall-overall': 0.5278350515463917,
 'f1-measure-overall': 0.5778781038374222,
 'loss': 0.25075810083321165}

In [25]:
predictor = SentenceTaggerPredictor(model, reader)
preds = predictor.predict('We are living in New York')

In [28]:
print(list(zip(preds['words'], preds['tags'])))

[('We', 'O'), ('are', 'O'), ('living', 'O'), ('in', 'O'), ('New', 'O'), ('York', 'O')]


## Predicting

### in conll format

In [43]:
from nltk import word_tokenize
def conll_sentence(labeled_sentence):
    ok_sentence = []
    for elem in labeled_sentence:
        new_word = []
        word, label = elem
        word_elems = word_tokenize(word)
        for word_elem in word_elems:
            if any(char.isalpha() for char in word_elem):
                new_word.append((word_elem, label))
            else:
                new_word.append((word_elem, 'O'))
        ok_sentence += new_word
    return ok_sentence

In [44]:
import pandas as pd
from ast import literal_eval
trial = pd.read_csv('data/tsd_trial.csv')
trial['spans'] = trial.spans.apply(literal_eval)

In [45]:
conll_preds = []
for text in trial.text:
    preds = predictor.predict(text)
    raw_pred = list(zip(preds['words'], preds['tags']))
    conll_pred = conll_sentence(raw_pred)
    conll_preds.append(conll_pred)

In [46]:
with open('./data/preds_trial.conll2003', 'w') as f:
    for sentence in conll_preds:
        for comb in sentence:
            f.write(comb[0] + ' ' + comb[1] + '\n')
        f.write('\n')